In [1]:
import torch
import argparse
import time
import math
import os, sys
import itertools
import pickle
from tqdm import tqdm
import numpy as np
import os.path as osp

In [2]:
torch.cuda.device_count()

1

In [3]:
def loadpkl(path):
    with open(path, 'rb') as f:
        rst=pickle.load(f)
    return rst

In [4]:
path="/share/data/mei-work/kangrui/github/ssref/data/refsum-data/arxiv-aiml/full_id_abs_title_cite_ref_author.pkl"
full_data=loadpkl(path)

In [42]:
path="/share/data/mei-work/kangrui/github/ssref/result/sentence-transformers_all-MiniLM-L6-v2/2023-04-03T07-59-29/test_result/eval_key_id2embed.pkl"
key_embedding=loadpkl(path)

In [43]:
print(len(key_embedding))

2037520


In [44]:
path="/share/data/mei-work/kangrui/github/ssref/result/sentence-transformers_all-MiniLM-L6-v2/2023-04-03T07-59-29/test_result/eval_test_id2embed.pkl"
query_embedding=loadpkl(path)

In [45]:
print(len(query_embedding))

968


In [9]:
path="/share/data/mei-work/kangrui/github/ssref/result/pretrained_pair_sbert/f1_result/eval_test_pred_1000.pkl"
pred_set=loadpkl(path)

In [40]:
path="/share/data/mei-work/kangrui/github/ssref/result/pretrained_pair_sbert/f1_result/eval_test_gold.pkl"
gold_set=loadpkl(path)

In [46]:
query_id2embedding=query_embedding
key_id2embedding=key_embedding

In [20]:
new_gold={}
for k,v in gold_set.items():
    newv=[]
    for paper in v:
        if paper in full_data.keys() and len(full_data[paper]['abstract'])>0:
            newv.append(paper)
    new_gold[k]=newv

In [21]:
gold_set=new_gold

In [48]:
score=0
total=0
for k,v in gold_set.items():
    for pid in v:
        if k in query_id2embedding.keys() and pid in key_id2embedding.keys():
            score+=query_id2embedding[k]@key_id2embedding[pid].T
            total+=1
print(score/total)

5.928945695397566


In [51]:
candidate_set={}
for k,v in pred_set.items():
    cur=set()
    for paper in v[:200]:
        cur.add(paper)
        for ref in full_data[paper]["references"]:
            cur.add(ref)
    candidate_set[k]=list(cur)

In [52]:
print(len(candidate_set))
total_len=0
for k,v in candidate_set.items():
    total_len+=len(v)
print(total_len)

944
3837147


In [53]:
def rerank(candidate_set,query_id2embedding,key_id2embedding,rerank_num=None,device=None):
    cnt=0
    total_mean=0
    mean_score=0
    pbar = tqdm(range(0,len(candidate_set),1), postfix=f"mean score:{mean_score}")
    query=list(candidate_set.keys())
    empty_embed=torch.tensor(np.zeros_like(next(iter(key_id2embedding.values()))))
    empty_embed_cnt=0
    if rerank_num is None:
        for v in candidate_set.values():
            if rerank_num is None or rerank_num>len(v):
                rerank_num=len(v)
                
    rerank_candidate={}   
    for i in pbar:
        query_paper=query[i]
        query_embedding=torch.tensor(query_id2embedding[query_paper]).unsqueeze(0)
        key_embedding=[]
        for paper in candidate_set[query_paper]:
            try:
                key_embedding.append(torch.tensor(key_id2embedding[paper]))
            except:
                key_embedding.append(empty_embed)
                empty_embed_cnt+=1
        
        key_embedding=torch.stack(key_embedding)
        query_embedding=query_embedding.to(device)
        key_embedding=key_embedding.to(device)
        pred_logits=torch.mm(query_embedding,key_embedding.T)
        mean_score=torch.mean(pred_logits[0])
        total_mean+=mean_score
        top_k=torch.topk(pred_logits[0],k=rerank_num)[1]
#         print(pred_logits[0][top_k[-5:]])
        rerank=[candidate_set[query_paper][idx.cpu().item()] for idx in top_k]
        rerank_candidate[query_paper]=rerank
        pbar.postfix=f"mean score:{mean_score}"
    print("total_mean",total_mean/len(pbar))
    print("empty embedding cnt->",empty_embed_cnt)
    print("min rerank num->",rerank_num)
    return rerank_candidate
        
        


In [54]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [55]:
rerank_candidate=rerank(candidate_set,query_id2embedding,key_id2embedding,device=device)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 944/944 [01:04<00:00, 14.69it/s, mean score:4.391404628753662]


total_mean tensor(3.7915, device='cuda:0')
empty embedding cnt-> 576703
min rerank num-> 1891


In [56]:
sys.path.append("/share/data/mei-work/kangrui/github/ssref")
from utils.test_utils import *
from utils.utils import *

In [57]:
def show_rerank_rst(rerank_candidate,gold_set):
    pool_length=len(next(iter(rerank_candidate.values())))
    i=2
    while 1:
        all_pred_topN=[]
        all_gold=[]
        for query_paper,paperlist in rerank_candidate.items():
            topN=paperlist[:i]
            all_pred_topN.append(topN)
            all_gold.append(gold_set[query_paper])
        all_pred_topN=np.array(all_pred_topN)
        metric = get_precision_recall_f1(all_pred_topN, all_gold)
        print(f"top{i}:\n{metric}")
        if i>pool_length:
            break
        i*=2

In [58]:
show_rerank_rst(rerank_candidate,gold_set)

top2:
{'precision': 0.22828389830508475, 'recall': 0.011212278876170656, 'f1': 0.021374637988312135, 'true_pos': 431, 'false_pos': 1457, 'false_neg': 38009, 'num_sample': 944}
top4:
{'precision': 0.19491525423728814, 'recall': 0.019146722164412072, 'f1': 0.0348681334924899, 'true_pos': 736, 'false_pos': 3040, 'false_neg': 37704, 'num_sample': 944}
top8:
{'precision': 0.1637976694915254, 'recall': 0.032180020811654525, 'f1': 0.05379168934115761, 'true_pos': 1237, 'false_pos': 6315, 'false_neg': 37203, 'num_sample': 944}
top16:
{'precision': 0.13453389830508475, 'recall': 0.05286160249739855, 'f1': 0.07589978920817275, 'true_pos': 2032, 'false_pos': 13072, 'false_neg': 36408, 'num_sample': 944}
top32:
{'precision': 0.10464115466101695, 'recall': 0.08223204994797087, 'f1': 0.09209250334686626, 'true_pos': 3161, 'false_pos': 27047, 'false_neg': 35279, 'num_sample': 944}
top64:
{'precision': 0.07877052436440678, 'recall': 0.12380332986472424, 'f1': 0.09628098461296158, 'true_pos': 4759, 'fa